In [2]:
import os
import sys
import time
import random
import pygame
import cv2
import mediapipe as mp
import math

# Constants and Settings
WINDOW_NAME = "Candy Collector"
GAME_TITLE = WINDOW_NAME

SCREEN_WIDTH, SCREEN_HEIGHT = 1200, 700
FPS = 90

HAND_SIZE = 150
HAND_HITBOX_SIZE = (60, 80)
CANDY_SIZES = (50, 50)
CANDY_SIZE_RANDOMIZE = (1, 2)
BONUS_CANDY_SIZES = (50, 50)
BONUS_CANDY_SIZE_RANDOMIZE = (1.2, 1.5)
CANDY_GENERATION_PROBABILITY = 0.1

DRAW_HITBOX = False
ANIMATION_SPEED = 0.08
BUTTONS_SIZES = (240, 90)

GAME_DURATION = 60
CANDY_SPAWN_TIME = 2
CANDY_MOVE_SPEED = {"min": (4, 4), "max": (15, 15)}
BONUS_PENALTY = 1

GOAL_SCORE = 100
BONUS_TIME_GAIN = 5

COLORS = {
    "title": (38, 61, 39),
    "score": (38, 61, 39),
    "timer": (38, 61, 39),
    "buttons": {
        "default": (56, 67, 209),
        "second": (87, 99, 255),
        "text": (255, 255, 255),
        "shadow": (46, 54, 163)
    }
}

MUSIC_VOLUME = 0.1
SOUNDS_VOLUME = 0.5

pygame.font.init()
FONTS = {
    "small": pygame.font.Font(None, 40),
    "medium": pygame.font.Font(None, 72),
    "big": pygame.font.Font(None, 120)
}

# Tips Text
tips_text = [
    "Instructions:",
    " You have 60 seconds to finish this game.",
    " Collect candy by hovering your hand over them.",
    " To win you need to score 100 points before time runs out",
    " Use Golden candies to get extra 5 seconds on your time.",
    " Avoid Red candies or it will deduct 3 seconds from your time.",
    " You have an opponent who shares the candies so try to move faster!",
    " GOOD LUCK!"
]

# Helper Functions
def load_image(img_path, size="default", convert="alpha", flip=False):
    if convert == "alpha":
        img = pygame.image.load(img_path).convert_alpha()
    else:
        img = pygame.image.load(img_path).convert()

    if flip:
        img = pygame.transform.flip(img, True, False)

    if size != "default":
        img = pygame.transform.smoothscale(img, size)

    return img

def draw_image(surface, img, pos, pos_mode="top_left"):
    if pos_mode == "center":
        pos = list(pos)
        pos[0] -= img.get_width() // 2
        pos[1] -= img.get_height() // 2
    surface.blit(img, pos)

def draw_text(surface, text, pos, color, font, pos_mode="top_left", shadow=False, shadow_color=(0, 0, 0), shadow_offset=2):
    label = font.render(text, 1, color)
    label_rect = label.get_rect()
    if pos_mode == "top_left":
        label_rect.x, label_rect.y = pos
    elif pos_mode == "center":
        label_rect.center = pos

    if shadow:
        label_shadow = font.render(text, 1, shadow_color)
        surface.blit(label_shadow, (label_rect.x - shadow_offset, label_rect.y + shadow_offset))

    surface.blit(label, label_rect)

def draw_tips(surface, tips_text, duration, width=800, height=400):
    pygame.time.set_timer(pygame.USEREVENT, duration * 1000)
    rect = pygame.Rect((SCREEN_WIDTH // 2 - width // 2, SCREEN_HEIGHT // 2 - height // 2), (width, height))
    pygame.draw.rect(surface, (255, 255, 255), rect)
    pygame.draw.rect(surface, (0, 0, 0), rect, 5)

    y_offset = rect.top + 20
    for line in tips_text:
        draw_text(surface, line, (SCREEN_WIDTH // 2, y_offset), (0, 0, 0), FONTS["small"], pos_mode="center")
        y_offset += 40

def button(surface, pos_y, text=None, click_sound=None):
    rect = pygame.Rect((SCREEN_WIDTH // 2 - BUTTONS_SIZES[0] // 2, pos_y), BUTTONS_SIZES)
    on_button = rect.collidepoint(pygame.mouse.get_pos())
    color = COLORS["buttons"]["second"] if on_button else COLORS["buttons"]["default"]

    pygame.draw.rect(surface, COLORS["buttons"]["shadow"], (rect.x - 6, rect.y - 6, rect.w, rect.h))
    pygame.draw.rect(surface, color, rect)
    if text:
        draw_text(surface, text, rect.center, COLORS["buttons"]["text"], FONTS["medium"], pos_mode="center", shadow=True, shadow_color=COLORS["buttons"]["shadow"])

    if on_button and pygame.mouse.get_pressed()[0]:
        if click_sound:
            click_sound.play()
        return True

# Classes
class Background:
    def __init__(self, image_path):
        self.image = load_image(image_path, size=(SCREEN_WIDTH, SCREEN_HEIGHT), convert="default")

    def draw(self, surface):
        draw_image(surface, self.image, (SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2), pos_mode="center")

class Candy:
    def __init__(self):
        size = (int(CANDY_SIZES[0] * random.uniform(*CANDY_SIZE_RANDOMIZE)),
                int(CANDY_SIZES[1] * random.uniform(*CANDY_SIZE_RANDOMIZE)))
        self.rect = pygame.Rect(random.randint(0, SCREEN_WIDTH - size[0]),
                                -size[1],
                                size[0], size[1])
        self.image = load_image("Candy/candyy.png", size=size)
        self.velocity = [random.uniform(*CANDY_MOVE_SPEED["min"]),
                         random.uniform(*CANDY_MOVE_SPEED["max"])]
        self.is_collected = False

    def move(self):
        self.rect.y += self.velocity[1]
        if self.rect.top > SCREEN_HEIGHT:
            self.rect.y = -self.rect.height

    def draw(self, surface):
        if not self.is_collected:
            draw_image(surface, self.image, self.rect.topleft)

class BonusCandy(Candy):
    def __init__(self):
        size = (int(BONUS_CANDY_SIZES[0] * random.uniform(*BONUS_CANDY_SIZE_RANDOMIZE)),
                int(BONUS_CANDY_SIZES[1] * random.uniform(*BONUS_CANDY_SIZE_RANDOMIZE)))
        self.rect = pygame.Rect(random.randint(0, SCREEN_WIDTH - size[0]),
                                -size[1],
                                size[0], size[1])
        self.image = load_image("Candy/Gcandy.png", size=size)
        self.velocity = [random.uniform(*CANDY_MOVE_SPEED["min"]),
                         random.uniform(*CANDY_MOVE_SPEED["max"])]
        self.is_collected = False

class HandTracking:
    def __init__(self):
        self.hand_tracking = mp.solutions.hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)
        self.hand_x = 0
        self.hand_y = 0
        self.results = None
        self.hand_closed = False

    def scan_hands(self, frame):
        image = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        self.results = self.hand_tracking.process(image)
        image.flags.writeable = True
        if self.results.multi_hand_landmarks:
            hand = self.results.multi_hand_landmarks[0]
            x_max = max([lm.x for lm in hand.landmark])
            x_min = min([lm.x for lm in hand.landmark])
            y_max = max([lm.y for lm in hand.landmark])
            y_min = min([lm.y for lm in hand.landmark])
            self.hand_closed = ((x_max - x_min) * (y_max - y_min)) < 0.1
            self.hand_x, self.hand_y = hand.landmark[9].x * SCREEN_WIDTH, hand.landmark[9].y * SCREEN_HEIGHT

class RedCandy(Candy):
    def __init__(self):
        size = (int(CANDY_SIZES[0] * random.uniform(*CANDY_SIZE_RANDOMIZE)),
                int(CANDY_SIZES[1] * random.uniform(*CANDY_SIZE_RANDOMIZE)))
        self.rect = pygame.Rect(random.randint(0, SCREEN_WIDTH - size[0]),
                                -size[1],
                                size[0], size[1])
        self.image = load_image("Candy/rcandy.png", size=size)
        self.velocity = [random.uniform(*CANDY_MOVE_SPEED["min"]),
                         random.uniform(*CANDY_MOVE_SPEED["max"])]
        self.is_collected = False

class Hand:
    def __init__(self):
        self.x, self.y = 0, 0
        self.left_click = False

    def update_position(self, x, y):
        self.x, self.y = x, y

    def collect_candies(self, candies, bonus_candies, red_candies, game_duration):
        collected_score = 0
        time_gained = 0
        candies_to_remove = []
        for candy in candies:
            if candy.rect.collidepoint(self.x, self.y) and not candy.is_collected:
                if isinstance(candy, RedCandy):
                    game_duration -= 3
                else:
                    collected_score += 1
                    if isinstance(candy, BonusCandy):
                        time_gained += BONUS_TIME_GAIN
                candy.is_collected = True
                candies_to_remove.append(candy)

        for candy in bonus_candies:
            if candy.rect.collidepoint(self.x, self.y) and not candy.is_collected:
                time_gained += BONUS_TIME_GAIN
                candy.is_collected = True
                candies_to_remove.append(candy)

        for candy in red_candies:
            if candy.rect.collidepoint(self.x, self.y) and not candy.is_collected:
                game_duration -= 3
                candy.is_collected = True
                candies_to_remove.append(candy)

        for candy in candies_to_remove:
            if candy in candies:
                candies.remove(candy)
            elif candy in bonus_candies:
                bonus_candies.remove(candy)
            elif candy in red_candies:
                red_candies.remove(candy)

        return collected_score, game_duration

    def draw(self, surface):
        pygame.draw.circle(surface, (237, 214, 110), (int(self.x), int(self.y)), HAND_SIZE // 2)
        if DRAW_HITBOX:
            pygame.draw.rect(surface, (255, 0, 0), (self.x - HAND_HITBOX_SIZE[0] // 2, self.y - HAND_HITBOX_SIZE[1] // 2, HAND_HITBOX_SIZE[0], HAND_HITBOX_SIZE[1]), 2)


class OpponentAI:
    def __init__(self):
        self.x, self.y = 0, 0
        self.target_candy = None
        self.velocity = [random.uniform(*CANDY_MOVE_SPEED["min"]), random.uniform(*CANDY_MOVE_SPEED["max"])]

    def choose_target_candy(self, candies, bonus_candies, red_candies):
        all_candies = candies + bonus_candies + red_candies
        if all_candies:
            if not self.target_candy or self.target_candy.is_collected:
                self.target_candy = random.choice(all_candies)
        else: 
            self.target_candy = None

    def move_towards_target(self):
        if self.target_candy:
            dx = self.target_candy.rect.centerx - self.x
            dy = self.target_candy.rect.centery - self.y
            dist = math.hypot(dx, dy)
            if dist != 0:
                self.velocity[0] = dx / dist
                self.velocity[1] = dy / dist
            self.x += self.velocity[0]
            self.y += self.velocity[1]

    def move(self):
        pass

    def collect_candies(self, candies, bonus_candies):
        pass

    def draw(self, surface):
        pygame.draw.circle(surface, (66, 245, 221), (int(self.x), int(self.y)), HAND_SIZE // 2)


def display_message(surface, text, pos):
    draw_text(surface, text, pos, COLORS["title"], FONTS["big"], pos_mode="center")

def restart_game():
    global score, start_time, game_duration, game_over
    score = 0
    start_time = time.time()
    game_duration = GAME_DURATION
    game_over = False

# Main Function
def main():
    pygame.init()
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    pygame.display.set_caption(GAME_TITLE)
    clock = pygame.time.Clock()

    bg = Background("Candy/cotton2.jpg")
    hand = Hand()
    hand_tracking = HandTracking()
    candies = []
    bonus_candies = []
    red_candies = []
    opponent = OpponentAI()  # Move this initialization here

    start_time = 0
    score = 0
    game_duration = GAME_DURATION
    game_over = False
    game_running = True
    paused = False  # Track if the game is paused
    game_active = False  # Track if the game is actively running

    cap = cv2.VideoCapture(0)
    
    click_sound = pygame.mixer.Sound("Assets/Sounds/music.wav")
    click_sound2 = pygame.mixer.Sound("Assets/Sounds/click.wav")
    
    while game_running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                game_running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    game_running = False
                elif event.key == pygame.K_p:  # Pause/unpause when 'P' is pressed
                    paused = not paused
                    if paused:
                        pygame.mixer.pause()
                        pygame.mixer.music.pause()
                        pygame.time.set_timer(pygame.USEREVENT, 0)
                    else:
                        pygame.mixer.unpause()
                        pygame.mixer.music.unpause()
                        pygame.time.set_timer(pygame.USEREVENT, 1000)

        if not game_active:
            screen.fill((0, 0, 0))
            bg.draw(screen)
            draw_text(screen, "Candy Collector", (SCREEN_WIDTH // 2, 100), COLORS["title"], FONTS["big"], pos_mode="center")
            if button(screen, 300, "Start!", click_sound):
                game_active = True
                start_time = time.time()
            if button(screen, 450, "Tips!", click_sound2):
                show_tips = True
                draw_tips(screen, tips_text, 3, width=1000, height=600)
                show_tips = False
            if button(screen, 600, "Exit", click_sound):
                game_running = False
            pygame.display.flip()
            continue

        if not paused:
            ret, frame = cap.read()
            if not ret:
                continue

            hand_tracking.scan_hands(frame)
            hand.update_position(hand_tracking.hand_x, hand_tracking.hand_y)
            hand.left_click = hand_tracking.hand_closed

            if len(candies) < 10:
                if random.random() < 0.05:
                    if random.random() < 0.1:
                        candies.append(RedCandy())
                    else:
                        candies.append(Candy())

            if len(bonus_candies) < 2:
                if random.random() < 0.01:
                    bonus_candies.append(BonusCandy())

            for candy in candies:
                candy.move()

            for bonus_candy in bonus_candies:
                bonus_candy.move()

            collected_score, game_duration = hand.collect_candies(candies, bonus_candies, red_candies, game_duration)
            score += collected_score
            
            opponent.choose_target_candy(candies, bonus_candies, red_candies)  # Choose target candy
            opponent.move_towards_target()  # Move towards target candy
            opponent.move()  # Move opponent
            opponent.collect_candies(candies, bonus_candies)  # Collect candies

        screen.fill((0, 0, 0))
        bg.draw(screen)

        for candy in candies:
            candy.draw(screen)

        for bonus_candy in bonus_candies:
            bonus_candy.draw(screen)

        hand.draw(screen)
        opponent.draw(screen)

        if not game_over and not paused:
            elapsed_time = time.time() - start_time
            draw_text(screen, f"Score: {score}", (20, 20), COLORS["score"], FONTS["small"])
            draw_text(screen, f"Time: {int(game_duration - elapsed_time)}", (SCREEN_WIDTH - 20, 20), COLORS["timer"], FONTS["small"], pos_mode="top_right")

            if score >= GOAL_SCORE:
                display_message(screen, "You win!", (SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2))
                game_over = True
            elif elapsed_time >= game_duration:
                display_message(screen, "Game Over", (SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2))
                game_over = True

        pygame.display.flip()
        clock.tick(FPS)

        if game_over:
            pygame.time.delay(5000)
            game_running = False

    cap.release()
    pygame.quit()
    sys.exit()
   
   

if __name__ == "__main__":
    main()


SystemExit: 

In [1]:
import os
import sys
import time
import random
import pygame
import cv2
import mediapipe as mp
import math

# Constants
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
GAME_TITLE = "Candy Collector"
GAME_DURATION = 60  # in seconds
GOAL_SCORE = 100
FPS = 60
HAND_RADIUS = 20

# Colors
COLORS = {
    "title": (255, 255, 255),
    "score": (255, 255, 255),
    "timer": (255, 255, 255),
    "background": (0, 0, 0),
    "hand": (0, 255, 0),
    "candy": (255, 165, 0),
    "bonus_candy": (255, 0, 0),
    "red_candy": (139, 0, 0),
    "opponent": (0, 0, 255),
}

# Fonts
FONTS = {
    "small": None,
    "big": None,
}

# Function to initialize Pygame and fonts
def initialize():
    pygame.init()
    pygame.mixer.init()
    pygame.font.init()

    # Load fonts
    FONTS["small"] = pygame.font.Font(None, 36)  # Replace 'None' with your small font path if needed
    FONTS["big"] = pygame.font.Font(None, 72)   # Replace 'None' with your big font path if needed

# Classes

class Hand:
    def __init__(self):
        self.x = SCREEN_WIDTH // 2
        self.y = SCREEN_HEIGHT // 2
        self.radius = HAND_RADIUS
        self.left_click = False

    def update_position(self, x, y):
        self.x = x
        self.y = y

    def draw(self, screen):
        pygame.draw.circle(screen, COLORS["hand"], (self.x, self.y), self.radius)


class HandTracking:
    def __init__(self):
        self.hand_x = SCREEN_WIDTH // 2
        self.hand_y = SCREEN_HEIGHT // 2
        self.hand_closed = False

    def scan_hands(self, frame):
        self.hand_x = 400
        self.hand_y = 300
        self.hand_closed = False

    def move_towards_target(self):
        pass


class Background:
    def __init__(self, image_path):
        self.image = pygame.image.load(image_path)
        self.image = pygame.transform.scale(self.image, (SCREEN_WIDTH, SCREEN_HEIGHT))

    def draw(self, screen):
        screen.blit(self.image, (0, 0))


class Candy:
    def __init__(self):
        self.x = random.randint(0, SCREEN_WIDTH)
        self.y = random.randint(0, SCREEN_HEIGHT)
        self.radius = 10
        self.is_collected = False

    def move(self):
        self.y += 1
        if self.y > SCREEN_HEIGHT:
            self.y = 0
            self.x = random.randint(0, SCREEN_WIDTH)

    def draw(self, screen):
        pygame.draw.circle(screen, COLORS["candy"], (self.x, self.y), self.radius)


class BonusCandy(Candy):
    def __init__(self):
        super().__init__()
        self.radius = 15

    def draw(self, screen):
        pygame.draw.circle(screen, COLORS["bonus_candy"], (self.x, self.y), self.radius)


class RedCandy(Candy):
    def __init__(self):
        super().__init__()
        self.radius = 12

    def draw(self, screen):
        pygame.draw.circle(screen, COLORS["red_candy"], (self.x, self.y), self.radius)


class OpponentAI:
    def __init__(self):
        self.x = random.randint(0, SCREEN_WIDTH)
        self.y = random.randint(0, SCREEN_HEIGHT)
        self.radius = 20
        self.target_candy = None

    def choose_target_candy(self, candies, bonus_candies, red_candies):
        all_candies = candies + bonus_candies + red_candies
        if not self.target_candy or self.target_candy.is_collected:
            if all_candies:
                self.target_candy = random.choice(all_candies)
            else:
                self.target_candy = None

    def move_towards_target(self):
        if self.target_candy:
            dx = self.target_candy.x - self.x
            dy = self.target_candy.y - self.y
            dist = math.sqrt(dx**2 + dy**2)
            if dist > 0:
                self.x += dx / dist
                self.y += dy / dist

    def move(self):
        self.x += random.randint(-1, 1)
        self.y += random.randint(-1, 1)
        self.x = max(self.radius, min(SCREEN_WIDTH - self.radius, self.x))
        self.y = max(self.radius, min(SCREEN_HEIGHT - self.radius, self.y))

    def draw(self, screen):
        pygame.draw.circle(screen, COLORS["opponent"], (int(self.x), int(self.y)), self.radius)


# Functions

def draw_text(screen, text, pos, color, font, pos_mode="center"):
    text_surface = font.render(text, True, color)
    text_rect = text_surface.get_rect()
    if pos_mode == "center":
        text_rect.center = pos
    elif pos_mode == "top_right":
        text_rect.topright = pos
    screen.blit(text_surface, text_rect)

def display_message(screen, message, pos):
    draw_text(screen, message, pos, COLORS["title"], FONTS["big"], pos_mode="center")
    pygame.display.flip()

def button(screen, y, label, sound):
    draw_text(screen, label, (SCREEN_WIDTH // 2, y), (0, 0, 0), FONTS["small"], pos_mode="center")
    rect = pygame.Rect(0, y - 25, 150, 50)
    click, _, _ = pygame.mouse.get_pressed()
    if click:
        sound.play()
    return rect.collidepoint(pygame.mouse.get_pos())

def draw_tips(screen, tips_text, delay, width, height):
    for i, line in enumerate(tips_text):
        draw_text(screen, line, (SCREEN_WIDTH // 2, 180 + i * 30), (255, 0, 0), FONTS["small"], pos_mode="center")
        pygame.display.flip()
        pygame.time.delay(delay * 1000)

def handle_events(hand, game_active):
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            return False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                return False
            elif event.key == pygame.K_p:
                game_active = not game_active
                pygame.mixer.music.pause() if not game_active else pygame.mixer.music.unpause()
                pygame.time.set_timer(pygame.USEREVENT, 1000) if game_active else pygame.time.set_timer(pygame.USEREVENT, 0)
    return True

def main():
    initialize()  # Initialize Pygame and fonts
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    pygame.display.set_caption(GAME_TITLE)
    clock = pygame.time.Clock()

    bg = Background("Candy/cotton2.jpg")
    hand = Hand()
    hand_tracking = HandTracking()
    candies = []
    bonus_candies = []
    red_candies = []
    opponent = OpponentAI()

    start_time = 0
    score = 0
    game_duration = GAME_DURATION
    game_over = False
    game_running = True
    game_active = False
    show_tips = False

    cap = cv2.VideoCapture(0)
    
    click_sound = pygame.mixer.Sound("Assets/Sounds/music.wav")
    click_sound2= pygame.mixer.Sound("Assets/Sounds/click.wav")

    while game_running:
        if not handle_events(hand, game_active):
            break
        
        if not game_active:
            screen.fill((0, 0, 0))
            bg.draw(screen)
            draw_text(screen, "Candy Collector", (SCREEN_WIDTH // 2, 100), COLORS["title"], FONTS["big"], pos_mode="center")
            if button(screen, 300, "Start Game", click_sound):
                game_active = True
                start_time = time.time()
            if button(screen, 450, "Tips!", click_sound2):
                show_tips = True
                draw_tips(screen, tips_text, 3, width=1000, height=600)
                show_tips = False
            if button(screen, 600, "Exit", click_sound):
                game_running = False
            pygame.display.flip()
            continue
        
        elapsed_time = time.time() - start_time

        if elapsed_time >= game_duration:
            game_over = True

        if not game_over:
            if len(candies) < 5:
                candies.append(Candy())
            if len(bonus_candies) < 2:
                bonus_candies.append(BonusCandy())
            if len(red_candies) < 1:
                red_candies.append(RedCandy())

            for candy in candies[:]:
                candy.move()
                if math.sqrt((hand.x - candy.x)**2 + (hand.y - candy.y)**2) <= hand.radius + candy.radius and not candy.is_collected:
                    candy.is_collected = True
                    score += 1

            for bonus_candy in bonus_candies[:]:
                bonus_candy.move()
                if math.sqrt((hand.x - bonus_candy.x)**2 + (hand.y - bonus_candy.y)**2) <= hand.radius + bonus_candy.radius and not bonus_candy.is_collected:
                    bonus_candy.is_collected = True
                    score += 3

            for red_candy in red_candies[:]:
                red_candy.move()
                if math.sqrt((hand.x - red_candy.x)**2 + (hand.y - red_candy.y)**2) <= hand.radius + red_candy.radius and not red_candy.is_collected:
                    red_candy.is_collected = True
                    score -= 5

            opponent.choose_target_candy(candies, bonus_candies, red_candies)  # Choose target candy
            opponent.move_towards_target()  # Move towards target candy
            opponent.move()  # Move opponent

            screen.fill((0, 0, 0))
            bg.draw(screen)

            for candy in candies:
                candy.draw(screen)

            for bonus_candy in bonus_candies:
                bonus_candy.draw(screen)

            hand.draw(screen)
            opponent.draw(screen)

            if not game_over:
                elapsed_time = time.time() - start_time
                draw_text(screen, f"Score: {score}", (20, 20), COLORS["score"], FONTS["small"])
                draw_text(screen, f"Time: {int(game_duration - elapsed_time)}", (SCREEN_WIDTH - 20, 20), COLORS["timer"], FONTS["small"], pos_mode="top_right")

                if score >= GOAL_SCORE:
                    display_message(screen, "You win!", (SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2))
                    game_over = True
                elif elapsed_time >= game_duration:
                    display_message(screen, "Game Over", (SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2))
                    game_over = True

            pygame.display.flip()
            clock.tick(FPS)

            if game_over:
                pygame.time.delay(5000)
                game_running = False

    cap.release()
    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()


pygame 2.5.2 (SDL 2.28.3, Python 3.11.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

e:\china_2nd\Web dev\Projects\web_env\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
